In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

In [53]:
#use healed files attached

#df = pd.read_csv('BonusEquations.csv')
df = pd.read_csv('FeynmanEquations.csv')
df = df[df['Formula'].notnull()]

In [54]:
for index, row in df.iterrows():
    for i in range (1, int(row['# variables']+1)):
        row['Formula'] = row['Formula'].replace(row[f"v{i}_name"], f"x{i}")
    print(row['Number'], row['Formula'])

1.0 exp(-x1**2/2)/sqrt(2*pi)
2.0 exp(-(x2/x1)**2/2)/(sqrt(2*pi)*x1)
3.0 exp(-((x2-x3)/x1)**2/2)/(sqrt(2*pi)*x1)
4.0 sqrt((x2-x1)**2+(x4-x3)**2)
5.0 x3*x4*x5/((x5-x4)**2+(x7-x6)**2+(x9-x8)**2)
6.0 x1/sqrt(1-x2**2/x3**2)
7.0 x1*x4+x2*x5+x3*x6
8.0 x1*x2
10.0 x1*x2*x4/(4*pi*x3*x4**3)
11.0 x1*x3/(4*pi*x2*x3**3)
12.0 x1*x2
13.0 x1*(x2+x3*x4*sin(x5))
9.0 1/2*x1*(x2**2+x3**2+x4**2)
14.0 x5*x1*x2*(1/x4-1/x3)
15.0 x1*x2*x3
16.0 1/2*x1*x2**2
17.0 (x1-x2*x4)/sqrt(1-x2**2/x3**2)
18.0 (x4-x3*x1/x2**2)/sqrt(1-x3**2/x2**2)
19.0 x1*x2/sqrt(1-x2**2/x3**2)
20.0 (x3+x2)/(1+x3*x2/x1**2)
21.0 (x1*x3+x2*x4)/(x1+x2)
22.0 x1*x2*sin(x3)
23.0 x1*x2*x3*sin(x4)
24.0 1/2*x1*(x2**2+x3**2)*1/2*x4**2
25.0 x1/x2
26.0 arcsin(x1*sin(x2))
27.0 1/(1/x1+x3/x2)
28.0 x1/x2
29.0 sqrt(x1**2+x2**2-2*x1*x2*cos(x3-x4))
30.0 x1*sin(x3*x2/2)**2/sin(x2/2)**2
31.0 arcsin(x1/(x3*x2))
32.0 x1**2*x2**2/(6*pi*x3*x4**3)
33.0 (1/2*x1*x2*x3**2)*(8*pi*x4**2/3)*(x5**4/(x5**2-x6**2)**2)
34.0 x1*x2*x3/x4
35.0 x3/(1-x2/x1)
36.0 (1+x2/x1)/sqrt(1-x

In [38]:
df

,Filename,Number,Name,Eqn. No.,Output,Formula,# variables,v1_name,v1_low,v1_high,...,v7_high,v8_name,v8_low,v8_high,v9_name,v9_low,v9_high,v10_name,v10_low,v10_high
0,test_1,1.0,Rutherford scattering,1.0,A,(Z_1*Z_2*alpha*hbar*c/(4*E_n*sin(theta/2)**2))**2,7.0,Z_1,1.0,2.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,test_2,2.0,3.55 Goldstein,2.0,k,m*k_G/L**2*(1+sqrt(1+2*E_n*L**2/(m*k_G**2))*co...,6.0,m,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,test_3,3.0,3.64 Goldstein,3.0,r,d*(1-alpha**2)/(1+alpha*cos(theta1-theta2)),4.0,d,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,test_4,4.0,3.16 Goldstein,4.0,v,sqrt(2/m*(E_n-U-L**2/(2*m*r**2))),5.0,m,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,test_5,5.0,3.74 Goldstein,5.0,t,2*pi*d**(3/2)/sqrt(G*(m1+m2)),4.0,d,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,test_6,6.0,3.99 Goldstein,6.0,alpha,sqrt(1+2*epsilon**2*E_n*L**2/(m*(Z_1*Z_2*q**2)...,7.0,epsilon,1.0,3.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,test_7,7.0,Friedman Equation,7.0,H_G,sqrt(8*pi*G*rho/3-alpha*c**2/d**2),5.0,G,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,test_8,8.0,Compton Scattering,8.0,K,E_n/(1+E_n/(m*c**2)*(1-cos(theta))),4.0,E_n,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,test_9,9.0,Gravitational wave ratiated power,9.0,Pwr,-32/5*G**4/c**5*(m1*m2)**2*(m1+m2)/r**5,5.0,G,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,test_10,10.0,Relativistic aberation,10.0,theta1,arccos((cos(theta2)-v/c)/(1-v/c*cos(theta2))),3.0,c,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
